Pour une lecture améliorée, ce notebook est idéalement lu avec une extension qui gère les niveaux des titres Markdown.  
Par exemple avec l'extension 'Table of Contents' du module 'Nbextensions' de Jupyter.

# Mise en place du notebook

## Import des modules

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress as reg_lin
from mesfonctions import scatter_plot
from mesfonctions import arrondi
from mesfonctions import display_scores

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.compose import TransformedTargetRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor

from graphviz import Source
from sklearn.tree import export_graphviz


In [2]:
# pour développement only - cellule à supprimer du notebook final
import mesfonctions
from importlib import reload  
mesfonctions = reload(mesfonctions)
# from mesfonctions import xxx

## Gestion des paramètres d'affichage de Jupyter

In [140]:
# pour gérer l'affichage des dataframes pandas
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200

# pour gérer l'affichage des arrays numpy
# 230 sur ma config bureau, 160 sur ma config portable
np.set_printoptions(linewidth=160)

# affichage des objets sklearn
set_config(display='diagram')

# pour afficher les gros fichiers sur toute la largeur de l'écran
from IPython.core.display import HTML
HTML("<style>.container { width:95% }</style>")

## Fonctions en cours de développement (à déplacer ultérieurement dans un fichier .py externe)

## Chargement du pickle du df nettoyé

In [83]:
df1 = pd.read_pickle('df1_après_nettoyage_Projet_Seattle.pkl')

# Suppression des lignes pour lesquelles la variable 'SiteEUI(kBtu/sf)' n'est pas renseignée
df1 = df1.dropna(subset=['SiteEUI(kBtu/sf)']).copy()

# Troisième approche du problème : utilisation des surfaces des PropertyUseType, en regroupant les types rares

Exemple de modalité problématique dans la deuxième approche : 'Swimming Pool' est présent dans plusieurs types de propriétés, mais les GFA sont toutes de 0 pour cette modalité de propriété.

## Rechargement du pickle du df nettoyé

In [115]:
df2 = pd.read_pickle('df1_après_nettoyage_Projet_Seattle.pkl')

## Nombre de propriétés renseignées pour chaque type de surface

Liste qui contient les modalités :

In [116]:
liste_modalites_types_1_2_3 = list(df2['LargestPropertyUseType'].dropna().unique()) + list(
    df2['SecondLargestPropertyUseType'].dropna().unique()) + list(
        df2['ThirdLargestPropertyUseType'].dropna().unique())

In [117]:
liste_modalites_types_1_2_3 = sorted(list(set(liste_modalites_types_1_2_3)))   # suppression doublons
liste_modalites_types_1_2_3

['Adult Education',
 'Automobile Dealership',
 'Bank Branch',
 'Bar/Nightclub',
 'College/University',
 'Convenience Store without Gas Station',
 'Convention Center',
 'Courthouse',
 'Data Center',
 'Distribution Center',
 'Enclosed Mall',
 'Fast Food Restaurant',
 'Financial Office',
 'Fire Station',
 'Fitness Center/Health Club/Gym',
 'Food Sales',
 'Food Service',
 'Hospital (General Medical & Surgical)',
 'Hotel',
 'K-12 School',
 'Laboratory',
 'Library',
 'Lifestyle Center',
 'Manufacturing/Industrial Plant',
 'Medical Office',
 'Movie Theater',
 'Multifamily Housing',
 'Museum',
 'Non-Refrigerated Warehouse',
 'Office',
 'Other',
 'Other - Education',
 'Other - Entertainment/Public Assembly',
 'Other - Lodging/Residential',
 'Other - Mall',
 'Other - Public Services',
 'Other - Recreation',
 'Other - Restaurant/Bar',
 'Other - Services',
 'Other - Technology/Science',
 'Other - Utility',
 'Other/Specialty Hospital',
 'Parking',
 'Performing Arts',
 'Personal Services (Health/Bea

Dictionnaire qui contient le nb de propriétés pour un type de surface donnée :

In [118]:
# initialisation
dict_modalites_types_1_2_3 = {x: 0 for x in liste_modalites_types_1_2_3}

In [119]:
dict_modalites_types_1_2_3

{'Adult Education': 0,
 'Automobile Dealership': 0,
 'Bank Branch': 0,
 'Bar/Nightclub': 0,
 'College/University': 0,
 'Convenience Store without Gas Station': 0,
 'Convention Center': 0,
 'Courthouse': 0,
 'Data Center': 0,
 'Distribution Center': 0,
 'Enclosed Mall': 0,
 'Fast Food Restaurant': 0,
 'Financial Office': 0,
 'Fire Station': 0,
 'Fitness Center/Health Club/Gym': 0,
 'Food Sales': 0,
 'Food Service': 0,
 'Hospital (General Medical & Surgical)': 0,
 'Hotel': 0,
 'K-12 School': 0,
 'Laboratory': 0,
 'Library': 0,
 'Lifestyle Center': 0,
 'Manufacturing/Industrial Plant': 0,
 'Medical Office': 0,
 'Movie Theater': 0,
 'Multifamily Housing': 0,
 'Museum': 0,
 'Non-Refrigerated Warehouse': 0,
 'Office': 0,
 'Other': 0,
 'Other - Education': 0,
 'Other - Entertainment/Public Assembly': 0,
 'Other - Lodging/Residential': 0,
 'Other - Mall': 0,
 'Other - Public Services': 0,
 'Other - Recreation': 0,
 'Other - Restaurant/Bar': 0,
 'Other - Services': 0,
 'Other - Technology/Scien

In [120]:
def compte_type_propriete(categorie):
    for indice, modalite, surface in zip(df2[categorie].index,   # supprimer itérateur indice
                                         df2[categorie],
                                         df2[categorie+'GFA']):
        if ~np.isnan(surface):
            if surface != 0:
                dict_modalites_types_1_2_3[modalite] +=1
    
compte_type_propriete('LargestPropertyUseType')
compte_type_propriete('SecondLargestPropertyUseType')
compte_type_propriete('ThirdLargestPropertyUseType')


In [121]:
# Tri du résultat
{k: v for k, v in sorted(dict_modalites_types_1_2_3.items(), key=lambda item: item[1])}

{'Swimming Pool': 0,
 'Convention Center': 1,
 'Enclosed Mall': 1,
 'Fast Food Restaurant': 1,
 'Fire Station': 1,
 'Police Station': 1,
 'Residential Care Facility': 1,
 'Wholesale Club/Supercenter': 1,
 'Courthouse': 2,
 'Food Sales': 2,
 'Lifestyle Center': 2,
 'Other - Technology/Science': 2,
 'Vocational School': 2,
 'Convenience Store without Gas Station': 3,
 'Other - Utility': 3,
 'Prison/Incarceration': 3,
 'Adult Education': 4,
 'Bar/Nightclub': 4,
 'Library': 4,
 'Movie Theater': 4,
 'Other - Mall': 4,
 'Personal Services (Health/Beauty, Dry Cleaning, etc)': 4,
 'Urgent Care/Clinic/Other Outpatient': 4,
 'Museum': 5,
 'Other - Public Services': 5,
 'Other/Specialty Hospital': 5,
 'Performing Arts': 5,
 'Other - Lodging/Residential': 6,
 'Automobile Dealership': 7,
 'Strip Mall': 7,
 'Other - Education': 8,
 'Financial Office': 9,
 'Other - Restaurant/Bar': 9,
 'Food Service': 10,
 'Repair Services (Vehicle, Shoe, Locksmith, etc)': 10,
 'Hospital (General Medical & Surgical)'

## Feature engineering

Pour chaque modalité de PropertyUseType, on ajoute une colonne au dataframe df2.  
Cette colonne contiendra la superficie pour la modalité considérée.  
Cependant, les types de propriété avec moins de N occurences seront regroupés dans la variable 'Rare'.  

In [122]:
N = 12

##### Ajout de colonne à df2 pour chaque PropertyUseType avec >N occurences

In [123]:
# Création d'une liste contenant le nom des colonnes créés
liste_variables_PropertyUseTypes_GFA = []

for moda in liste_modalites_types_1_2_3:
    if dict_modalites_types_1_2_3[moda] > N :
        nom_moda_GFA = moda + ' GFA'
        df2[nom_moda_GFA] = 0   # initialisation à 0 sq/f
        liste_variables_PropertyUseTypes_GFA.append(nom_moda_GFA)
    
# Colonne 'Rare GFA'
liste_variables_PropertyUseTypes_GFA.append('Rare GFA')
df2['Rare GFA'] = 0

In [124]:
df2.sample(100)

,OSEBuildingID,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,Bank Branch GFA,College/University GFA,Data Center GFA,Distribution Center GFA,Fitness Center/Health Club/Gym GFA,Hotel GFA,K-12 School GFA,Laboratory GFA,Medical Office GFA,Multifamily Housing GFA,Non-Refrigerated Warehouse GFA,Office GFA,Other GFA,Other - Entertainment/Public Assembly GFA,Other - Recreation GFA,Other - Services GFA,Parking GFA,Refrigerated Warehouse GFA,Residence Hall/Dormitory GFA,Restaurant GFA,Retail Store GFA,Self-Storage Facility GFA,Senior Care Community GFA,Social/Meeting Hall GFA,Supermarket/Grocery Store GFA,Worship Facility GFA,Rare GFA
1281,21438,NonResidential,Small- and Mid-Sized Office,98109.0,7,LAKE UNION,47.620550,-122.338810,1999,1.0,4,38038,0,38038,Office,38038.0,Parking,19600.0,NaN,NaN,84.0,52.700001,53.599998,165.399994,168.399994,2.003808e+06,2.040142e+06,0.000000e+00,5.872824e+05,2003807.0,0.000000,0.0,13.97,0.37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1931,23877,SPS-District K-12,K-12 School,98108.0,2,GREATER DUWAMISH,47.552260,-122.296290,1971,1.0,2,52609,0,52609,K-12 School,52609.0,NaN,NaN,NaN,NaN,67.0,34.299999,38.000000,78.300003,83.400002,1.801942e+06,2.001526e+06,0.000000e+00,3.124751e+05,1066165.0,7357.766113,735777.0,46.51,0.88,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
176,283,NonResidential,Small- and Mid-Sized Office,98125.0,5,NORTH,47.718950,-122.294750,1972,1.0,5,50759,0,50759,Office,47079.0,Restaurant,3680.0,NaN,NaN,62.0,62.000000,65.400002,170.399994,174.100006,3.147042e+06,3.321734e+06,0.000000e+00,7.498631e+05,2558533.0,5885.089844,588509.0,49.09,0.97,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
568,762,NonResidential,Retail Store,98101.0,7,DOWNTOWN,47.609250,-122.337340,1924,1.0,3,77555,0,77555,Retail Store,73563.0,NaN,NaN,NaN,NaN,39.0,106.099998,106.099998,333.000000,333.000000,7.802206e+06,7.802206e+06,0.000000e+00,2.286650e+06,7802052.0,1.530000,153.0,54.40,0.70,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
385,540,NonResidential,Warehouse,98108.0,2,GREATER DUWAMISH,47.547690,-122.325100,1966,1.0,1,22780,0,22780,Non-Refrigerated Warehouse,22780.0,NaN,NaN,NaN,NaN,33.0,17.600000,17.600000,55.400002,55.400002,4.020513e+05,4.020513e+05,0.000000e+00,1.178345e+05,402051.0,0.000000,0.0,2.80,0.12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
912,20212,NonResidential,Mixed Use Property,98107.0,6,BALLARD,47.668400,-122.387880,1960,1.0,4,20539,0,20539,Retail Store,7387.0,Multifamily Housing,5264.0,Restaurant,3700.0,NaN,81.000000,81.800003,198.399994,199.300003,1.678459e+06,1.695248e+06,0.000000e+00,3.292575e+05,1123427.0,5550.319824,555032.0,37.31,1.82,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1161,21125,NonResidential,Small- and Mid-Sized Office,98103.0,6,LAKE UNION,47.651140,-122.349540,2001,1.0,5,33740,6010,27730,Office,15908.0,Retail Store,8762.0,Parking,1500.0,79.0,52.299999,52.299999,164.399994,164.399994,1.291405e+06,1.291405e+06,0.000000e+00,3.784892e+05,1291405.0,0.000000,0.0,9.00,0.27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2682,26593,NonResidential,Small- and Mid-Sized Office,98104.0,7,DOWNTOWN,47.601020,-122.333780,1900,1.0,4,22840,0,22840,Office,18400.0,Other,4440.0,NaN,NaN,64.0,38.599998,43.000000,121.199997,135.100006,8.813862e+05,9.830534e+05,0.000000e+00,2.583195e+05,881386.0,0.000000,0.0,6.14,0.27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
530,71

In [125]:
liste_variables_PropertyUseTypes_GFA

['Bank Branch GFA',
 'College/University GFA',
 'Data Center GFA',
 'Distribution Center GFA',
 'Fitness Center/Health Club/Gym GFA',
 'Hotel GFA',
 'K-12 School GFA',
 'Laboratory GFA',
 'Medical Office GFA',
 'Multifamily Housing GFA',
 'Non-Refrigerated Warehouse GFA',
 'Office GFA',
 'Other GFA',
 'Other - Entertainment/Public Assembly GFA',
 'Other - Recreation GFA',
 'Other - Services GFA',
 'Parking GFA',
 'Refrigerated Warehouse GFA',
 'Residence Hall/Dormitory GFA',
 'Restaurant GFA',
 'Retail Store GFA',
 'Self-Storage Facility GFA',
 'Senior Care Community GFA',
 'Social/Meeting Hall GFA',
 'Supermarket/Grocery Store GFA',
 'Worship Facility GFA',
 'Rare GFA']

##### Ajout des surfaces des PropertyUseType dans le dataframe df2

In [126]:
def ajout_surfaces_2(categorie):
    for indice, modalite, surface in zip(df2[categorie].index,
                                         df2[categorie],
                                         df2[categorie+'GFA']):
        if ~np.isnan(surface):
            if modalite + ' GFA' in liste_variables_PropertyUseTypes_GFA:
                df2.at[indice, modalite + ' GFA'] += surface
            else:
                df2.at[indice, 'Rare GFA'] += surface
    
ajout_surfaces_2('LargestPropertyUseType')
ajout_surfaces_2('SecondLargestPropertyUseType')
ajout_surfaces_2('ThirdLargestPropertyUseType')

In [127]:
df2.sample(100, random_state=0)

,OSEBuildingID,BuildingType,PrimaryPropertyType,ZipCode,CouncilDistrictCode,Neighborhood,Latitude,Longitude,YearBuilt,NumberofBuildings,NumberofFloors,PropertyGFATotal,PropertyGFAParking,PropertyGFABuilding(s),LargestPropertyUseType,LargestPropertyUseTypeGFA,SecondLargestPropertyUseType,SecondLargestPropertyUseTypeGFA,ThirdLargestPropertyUseType,ThirdLargestPropertyUseTypeGFA,ENERGYSTARScore,SiteEUI(kBtu/sf),SiteEUIWN(kBtu/sf),SourceEUI(kBtu/sf),SourceEUIWN(kBtu/sf),SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),TotalGHGEmissions,GHGEmissionsIntensity,Bank Branch GFA,College/University GFA,Data Center GFA,Distribution Center GFA,Fitness Center/Health Club/Gym GFA,Hotel GFA,K-12 School GFA,Laboratory GFA,Medical Office GFA,Multifamily Housing GFA,Non-Refrigerated Warehouse GFA,Office GFA,Other GFA,Other - Entertainment/Public Assembly GFA,Other - Recreation GFA,Other - Services GFA,Parking GFA,Refrigerated Warehouse GFA,Residence Hall/Dormitory GFA,Restaurant GFA,Retail Store GFA,Self-Storage Facility GFA,Senior Care Community GFA,Social/Meeting Hall GFA,Supermarket/Grocery Store GFA,Worship Facility GFA,Rare GFA
2738,26761,NonResidential,Small- and Mid-Sized Office,98105.0,4,LAKE UNION,47.653730,-122.327210,2007,1.0,3,44510,18228,26282,Office,26282.0,Parking,18228.0,NaN,NaN,23.0,83.300003,85.599998,261.399994,268.899994,2.188051e+06,2.250450e+06,0.000000e+00,6.412810e+05,2188051.00,0.000000,0.000000e+00,15.25,0.34,0,0,0,0,0,0,0,0,0,0,0,26282,0,0,0,0,18228,0,0,0,0,0,0,0,0,0,0
1839,23562,NonResidential,Mixed Use Property,98104.0,7,DOWNTOWN,47.601950,-122.335160,1913,1.0,3,43878,0,43878,Multifamily Housing,12000.0,Office,8262.0,Retail Store,6312.0,NaN,23.400000,23.400000,73.500000,73.500000,7.697713e+05,7.697713e+05,0.000000e+00,2.256070e+05,769771.00,0.000000,0.000000e+00,5.37,0.12,0,0,0,0,0,0,0,0,0,12000,0,8262,0,0,0,0,0,0,0,0,6312,0,0,0,0,0,0
1139,21088,NonResidential,Small- and Mid-Sized Office,98103.0,4,LAKE UNION,47.649040,-122.346090,1985,1.0,3,31417,8110,23307,Office,26295.0,Parking,12890.0,NaN,NaN,94.0,42.799999,46.200001,99.599998,102.099998,1.126157e+06,1.214409e+06,0.000000e+00,2.014866e+05,687472.00,4386.850098,4.386850e+05,28.09,0.89,0,0,0,0,0,0,0,0,0,0,0,26295,0,0,0,0,12890,0,0,0,0,0,0,0,0,0,0
1043,20613,NonResidential,Warehouse,98134.0,2,GREATER DUWAMISH,47.569820,-122.334790,1951,1.0,1,24000,0,24000,Non-Refrigerated Warehouse,20400.0,Office,3600.0,NaN,NaN,NaN,52.400002,62.900002,93.000000,104.099998,1.257034e+06,1.510110e+06,0.000000e+00,1.278410e+05,436193.00,8208.400391,8.208400e+05,46.64,1.94,0,0,0,0,0,0,0,0,0,0,20400,3600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2076,24374,NonResidential,Retail Store,98118.0,2,SOUTHEAST,47.565140,-122.287810,1968,1.0,1,36071,0,36071,Retail Store,36071.0,NaN,NaN,NaN,NaN,97.0,42.099998,42.099998,81.900002,81.900002,1.519952e+06,1.519952e+06,0.000000e+00,1.903151e+05,649355.00,8705.969727,8.705970e+05,50.76,1.41,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36071,0,0,0,0,0,0
921,20237,NonResidential,Retail Store,98105.0,4,NORTHEAST,47.660330,-122.313480,1900,1.0,3,32960,0,32960,Retail Store,32960.0,NaN,NaN,NaN,NaN,71.0,24.700001,24.700001,77.500000,77.500000,8.136410e+05,8.136410e+05,0.000000e+00,2.384645e+05,813641.00,0.000000,0.000000e+00,5.67,0.17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32960,0,0,0,0,0,0
444,605,NonResidential,Small- and Mid-Sized Office,98109.0,7,LAKE UNION,47.633760,-122.339680,1970,1.0,7,94002,0,94002,Office,93905.0,Parking,31348.0,NaN,NaN,70.0,60.299999,60.299999,189.300003,189.300003,5.662234e+06,5.662234e+06,0.000000e+00,1.659506e+06,5662234.00,0.000000,0.000000e+00,39.47,0.42,0,0,0,0,0,0,0,0,0,0,0,93905,0,0,0,0,31348,0,0,0,0,0,0,0,0,0,0
2411,25554,NonResidential,Other,98104.0,7,DOWNTOWN,47.599760,-122.335140,1900,1.0,3,22200,0,22200,Other,22200.0,NaN,NaN,NaN,NaN,NaN,8.500000,10.400000,15.000000,17.500000,1.887457e+05,2.318471e+05,0.000000e+00,1.889000e+04,64453.00,1242.929932,1.242930e+05,

## Train test split 

POUR LE NOTEBOOK FINAL IL FAUDRA FAIRE LE SPLIT PLUS TOT (avant le nettoyage)  
IL FAUDRA FAIRE LE MEME NETTOYAGE SUR LES JEUX TRAIN ET TEST

POUR L'INSTANT ON NE TRAVAILLE QU'A LA PREDICTION DE SiteEUI(kBtu/sf)

In [128]:
df2 = df2.dropna(subset=['SiteEnergyUse(kBtu)']).copy()

##### Variable à prédire :

In [129]:
y4 = df2[['SiteEnergyUse(kBtu)']].copy()

##### Variables explicatives :

In [130]:
X4 = df2[liste_variables_PropertyUseTypes_GFA].copy()

##### Split :

In [131]:
X_train_EU, X_test_EU, y_train_EU, y_test_EU = train_test_split(X4, y4, test_size=0.2, random_state=0)

## Régression linéaire simple, sans rescaling, cross-validation

In [132]:
lin_reg = LinearRegression();

In [133]:
lin_scores = cross_val_score(lin_reg, X_train_EU, y_train_EU,
                             scoring="neg_root_mean_squared_error", cv=10)
display_scores(-lin_scores, puissance10=6)

Scores: 7.02e6 8.80e6 11.90e6 6.20e6 10.04e6 6.30e6 16.50e6 6.98e6 22.97e6 12.10e6 
Mean: 10.88e6
Standard deviation: 5.36e6


In [134]:
lin_scores = cross_val_score(lin_reg, X_train_EU, y_train_EU,
                             scoring="r2", cv=10)
display_scores(lin_scores)

Scores: 0.747 0.243 0.805 0.497 0.888 0.685 0.953 0.556 0.667 0.362 
Mean: 0.64
Standard deviation: 0.227


On passe de RMSE=10.7e6 (baseline / N=-1) à 10.7e6 (N=10).  
On passe de R²=0.678 (baseline / N=-1) à :
- 0.650 (N=20),  
- 0.640 (N=15),  
- 0.640 (N=12),  
- 0.675 (N=10),  
- 0.672 (N=8),  
- 0.670 (N=5),  
- 0.676 (N=2),  
- 0.678 (N=1).
- 0.678 (N=0).

## Données recalibrées par StandardScaler

In [135]:
ss = StandardScaler()
X_train_EU_ss = ss.fit_transform(X_train_EU)
X_test_EU_ss = ss.transform(X_test_EU)

### Régression Ridge

In [136]:
ridge_ss = RidgeCV(alphas=np.logspace(-2, 2, 100), cv=10)    # 100 valeurs alpha entre 1e0 et 1e2
ridge_ss.fit(X_train_EU_ss, y_train_EU)

RidgeCV(alphas=array([1.00000000e-02, 1.09749877e-02, 1.20450354e-02, 1.32194115e-02, 1.45082878e-02, 1.59228279e-02, 1.74752840e-02, 1.91791026e-02, 2.10490414e-02,
       2.31012970e-02, 2.53536449e-02, 2.78255940e-02, 3.05385551e-02, 3.35160265e-02, 3.67837977e-02, 4.03701726e-02, 4.43062146e-02, 4.86260158e-02,
       5.33669923e-02, 5.85702082e-02, 6.42807312e-02, 7.05480231e-0...
       8.11130831e+00, 8.90215085e+00, 9.77009957e+00, 1.07226722e+01, 1.17681195e+01, 1.29154967e+01, 1.41747416e+01, 1.55567614e+01, 1.70735265e+01,
       1.87381742e+01, 2.05651231e+01, 2.25701972e+01, 2.47707636e+01, 2.71858824e+01, 2.98364724e+01, 3.27454916e+01, 3.59381366e+01, 3.94420606e+01,
       4.32876128e+01, 4.75081016e+01, 5.21400829e+01, 5.72236766e+01, 6.28029144e+01, 6.89261210e+01, 7.56463328e+01, 8.30217568e+01, 9.11162756e+01,
       1.00000000e+02]),
        cv=10)

In [137]:
best_alpha = ridge_ss.alpha_
best_alpha

0.01

In [138]:
# R²
R2_ss_ridge_cv = arrondi(ridge_ss.best_score_)
R2_ss_ridge_cv

0.64

On passe de 0.680 (baseline / N=-1) à :
- 0.650 (N=20).  
- 0.640 (N=15).  
- 0.676 (N=10).  
- 0.640 (N=12). 
- 0.673 (N=8).  
- 0.671 (N=5).
- 0.678 (N=2).  
- 0.680 (N=1).  
- 0.680 (N=0).  

##### R² sur le test set avec le meilleur alpha issu de la CV

In [139]:
ridge_ss = Ridge(alpha=best_alpha)
ridge_ss.fit(X_train_EU_ss, y_train_EU)
print(arrondi(ridge_ss.score(X_test_EU_ss, y_test_EU)))

0.801


On passe de 0.815 (baseline / N=-1) à :
- 0.773 (N=20).  
- 0.801 (N=15). 
- 0.801 (N=12).
- 0.84  (N=10).
- 0.838 (N=8).  
- 0.838 (N=5).  
- 0.836 (N=2).  
- 0.816 (N=1).  
- 0.815 (N=0).  

##### Conclusion sur la 3è approche :

L'utilisation des surfaces des PropertyUseType, en regroupant les types rares implique un effet défavorable sur le train set et favorable sur le test set.